In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import numpy.fft as fft
from scipy import signal as scisig
from viterbi_utils import *
from hmmlearn.hmm import GaussianHMM

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)

In [2]:
df_train = pd.read_pickle('../features/train_clean.pkl')
df_test = pd.read_pickle('../features/test_clean.pkl')
TARGET = "open_channels"
df_test[TARGET] = 0

df_train["group"] = df_train["batch"].astype("str") + "_" + df_train["mini_batch"].astype("str")
df_test["group"] = df_test["batch"].astype("str") + "_" + df_test["mini_batch"].astype("str")

df_train["signal_original"] = df_train["signal"].copy()
df_test["signal_original"] = df_test["signal"].copy()

print(f"train size:{df_train.shape}, test size:{df_test.shape}")
df_train.head()

train size:(4500000, 8), test size:(2000000, 9)


,time,signal,open_channels,local_time,batch,mini_batch,group,signal_original
0,0.0001,-2.7600,0,0.0001,1,1,1_1,-2.7600
1,0.0002,-2.8557,0,0.0002,1,1,1_1,-2.8557
2,0.0003,-2.4074,0,0.0003,1,1,1_1,-2.4074
3,0.0004,-3.1404,0,0.0004,1,1,1_1,-3.1404
4,0.0005,-3.1525,0,0.0005,1,1,1_1,-3.1525


In [3]:
# mini model
BATCH_GROUP = [6, 9]
df_train = df_train[df_train.batch.isin(BATCH_GROUP)].reset_index(drop=True)
TEST_GROUP = ["1_3", "2_2"]
df_test = df_test[df_test.group.isin(TEST_GROUP)].reset_index(drop=True)

print(f"train size:{df_train.shape}, test size:{df_test.shape}")

train size:(1000000, 8), test size:(200000, 9)


In [4]:
# remove the 50 hz noise using notch filter
for group_i in df_train.group.unique():

    batch_i = df_train[df_train.group.isin([group_i])]
    signal_recovered = rm_noise(batch_i, Q=60)
    df_train.loc[df_train.group.isin([group_i]), "signal"] = signal_recovered

In [5]:
print(df_train.shape)
df_train.head()

(1000000, 8)


,time,signal,open_channels,local_time,batch,mini_batch,group,signal_original
0,250.000107,2.880314,5,0.000107,6,1,6_1,2.8555
1,250.000198,3.114359,5,0.000198,6,1,6_1,3.0907
2,250.000305,3.550181,5,0.000305,6,1,6_1,3.5277
3,250.000397,4.003481,5,0.000397,6,1,6_1,3.9822
4,250.000504,3.356860,5,0.000504,6,1,6_1,3.3368


In [6]:
signals = df_train.signal.values
channels = df_train.open_channels.values

print(signals.shape, channels.shape)

(1000000,) (1000000,)


In [70]:
class GaussHMM:
    def __init__(self, init):
        self.init = init

    def fit(self, signals, channels):

        self.hmm = GaussianHMM(n_components=len(self.init), covariance_type="full", n_iter=100)
        self.hmm.fit(signals.reshape([-1, 1])[:100])
        self.hmm.means_ = self.get_mean(signals, channels)
        self.hmm.covars_ = self.get_cov(signals, channels)
        self.hmm.startprob_ = [0, 0, 0, 0, 0, 1]
        self.hmm.transmat_ = self.markov_p_trans(channels)
        
    def predict(self, signals):
        pred = self.model.hmm.predict(signals.reshape([-1, 1]))
        return pred
    
    def predict_proba(self, signals):
        prob = self.model.hmm.predict_proba(signals.reshape([-1, 1])).round(3)
        return prob
        
    def get_mean(self, signals, channels):

        sig_mean = []
        for chan_i in range(len(np.unique(channels))):
            sig_mean.append(signals[channels == chan_i].mean())

        return np.array(sig_mean).reshape([-1, 1])

    def get_cov(self, signals, channels):

        sig_cov = []
        for chan_i in range(len(np.unique(channels))):
            sig_cov.append(np.cov(signals[channels == chan_i]))

        return np.array(sig_cov).reshape([-1, 1, 1])
    
    def markov_p_trans(self, states):
        max_state = np.max(states)
        states_next = np.roll(states, -1)
        matrix = []
        for i in range(max_state + 1):
            current_row = np.histogram(states_next[states == i], bins=np.arange(max_state + 2))[0]
            if np.sum(current_row) == 0: # if a state doesn't appear in states...
                current_row = np.ones(max_state + 1) / (max_state + 1) # ...use uniform probability
            else:
                current_row = current_row / np.sum(current_row) # normalize to 1
            matrix.append(current_row)
        return np.array(matrix)

In [79]:
init_prob = [0,0,0,0,0,1]
ghmm = GaussHMM(init_prob)

In [80]:
ghmm.fit(signals, channels)

In [81]:
pred= ghmm.hmm.predict(signals.reshape([-1, 1]))

In [82]:
prob= ghmm.hmm.predict_proba(signals.reshape([-1, 1])).round(3)

In [83]:
prob

array([[0.   , 0.   , 0.   , 0.   , 0.   , 1.   ],
       [0.   , 0.   , 0.   , 0.   , 0.001, 0.999],
       [0.   , 0.   , 0.   , 0.   , 0.   , 1.   ],
       ...,
       [0.   , 0.   , 0.   , 0.007, 0.993, 0.   ],
       [0.   , 0.   , 0.   , 0.2  , 0.8  , 0.   ],
       [0.   , 0.   , 0.001, 0.999, 0.   , 0.   ]])